# Deploying Mercury-Coder using AWS Jumpstart

This notebook shows how you can deploy Mercury Coder as a Sagemaker Endpoint with AWS Jumpstart. 

## Prerequisites

1. This notebook must be opened from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio 
2. Ensure that your IAM role has AmazonSageMakerFullAccess

**Note**: It is possible to run this notebook on a local machine if you manually set the `role_arn` to an IAM role with AmazonSagemakerFullAccess permissions.

## Step 1: Subscribe to the model package

To subscribe to Mercury-Coder:

1. Open the Mercury listing: [Mercury Coder - Sagemaker Marketplace](https://aws.amazon.com/marketplace/pp/prodview-qnkxatxcm4l7o)
2. Click on **Continue to subscribe**.
3. Accept the offer on the next page after reviewing the EULA, pricing and support terms.

## Step 2: Create an endpoint

In [1]:
# import necessary packages and set the correcr region and execution role

import sagemaker
from sagemaker import ModelPackage, get_execution_role
import boto3
from sagemaker.utils import name_from_base
from time import perf_counter

region = boto3.Session().region_name
role_arn = get_execution_role()
sagemaker_session = sagemaker.Session()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
# Set the correct model package ARN corresponding to your region

model_package_map = {
    "us-east-1": "arn:aws:sagemaker:us-east-1:865070037744:model-package/mercury-coder-7990ba9bf79b3bb29597dd86491bb42a",
    "us-east-2": "arn:aws:sagemaker:us-east-2:057799348421:model-package/mercury-coder-7990ba9bf79b3bb29597dd86491bb42a",
    "us-west-1": "arn:aws:sagemaker:us-west-1:382657785993:model-package/mercury-coder-7990ba9bf79b3bb29597dd86491bb42a",
    "us-west-2": "arn:aws:sagemaker:us-west-2:594846645681:model-package/mercury-coder-7990ba9bf79b3bb29597dd86491bb42a",
    "ca-central-1": "arn:aws:sagemaker:ca-central-1:470592106596:model-package/mercury-coder-7990ba9bf79b3bb29597dd86491bb42a",
    "ca-central-1": "arn:aws:sagemaker:ca-central-1:470592106596:model-package/mercury-coder-7990ba9bf79b3bb29597dd86491bb42a",
    "eu-west-1": "arn:aws:sagemaker:eu-west-1:985815980388:model-package/mercury-coder-7990ba9bf79b3bb29597dd86491bb42a",
    "eu-west-2": "arn:aws:sagemaker:eu-west-2:856760150666:model-package/mercury-coder-7990ba9bf79b3bb29597dd86491bb42a",
    "eu-west-3": "arn:aws:sagemaker:eu-west-3:843114510376:model-package/mercury-coder-7990ba9bf79b3bb29597dd86491bb42a",
    "eu-north-1": "arn:aws:sagemaker:eu-north-1:136758871317:model-package/mercury-coder-7990ba9bf79b3bb29597dd86491bb42a",
    "ap-southeast-1": "arn:aws:sagemaker:ap-southeast-1:192199979996:model-package/mercury-coder-7990ba9bf79b3bb29597dd86491bb42a",
    "ap-southeast-2": "arn:aws:sagemaker:ap-southeast-2:666831318237:model-package/mercury-coder-7990ba9bf79b3bb29597dd86491bb42a",
    "ap-northeast-2": "arn:aws:sagemaker:ap-northeast-2:745090734665:model-package/mercury-coder-7990ba9bf79b3bb29597dd86491bb42a",
    "ap-northeast-1": "arn:aws:sagemaker:ap-northeast-1:977537786026:model-package/mercury-coder-7990ba9bf79b3bb29597dd86491bb42a",
    "ap-south-1": "arn:aws:sagemaker:ap-south-1:077584701553:model-package/mercury-coder-7990ba9bf79b3bb29597dd86491bb42a",
    "sa-east-1": "arn:aws:sagemaker:sa-east-1:270155090741:model-package/mercury-coder-7990ba9bf79b3bb29597dd86491bb42a",
}

if region not in model_package_map:
    raise Exception("UNSUPPORTED REGION")

package_arn = model_package_map[region]

In [3]:
# Create the model package

endpoint_name = name_from_base("mercury-coder-endpoint")  # set this to your liking
model = ModelPackage(
    role=role_arn, model_package_arn=package_arn, sagemaker_session=sagemaker_session
)

In [4]:
# Deploy the Model. This may take 5-10 minutes to run

instance_type = "ml.p5.48xlarge"  # We only support ml.p5.48xlarge instances at the moment
start = perf_counter()
deployed_model = model.deploy(
    initial_instance_count=1, instance_type=instance_type, endpoint_name=endpoint_name
)
print(f"\nDeployment took {perf_counter() - start:.2f} seconds")

-----------------!
Deployment took 542.84 seconds


## Step 3: Run real-time inference on the new endpoint

In [5]:
# Create a simple Sagemaker predictor

predictor = sagemaker.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=sagemaker.serializers.JSONSerializer(),
    deserializer=sagemaker.deserializers.JSONDeserializer(),
)

In [16]:
# Run inference

payload = {
    "messages": [
        {
            "role": "user",
            "content": "Write a pomodoro app backend in python",
        },
    ],
    "max_tokens": 2048,
    "model": "mercury-coder"
}
start = perf_counter()
outputs = predictor.predict(payload)
eta = perf_counter() - start
print(f"Speed: {outputs['usage']['completion_tokens'] / eta:.2f} tokens / second\n")
print("=" * 60)
print(f"\n{outputs["choices"][0]["message"]["content"]}")

Speed: 666.74 tokens / second


Certainly! Below is a simple implementation of a Pomodoro Timer backend in Python. This backend will handle the core functionality of a Pomodoro Timer, including starting, stopping, and managing the timer.

### Pomodoro Timer Backend in Python

```python
import time
import threading

class PomodoroTimer:
    def __init__(self, work_duration=25, short_break_duration=5, long_break_duration=15, long_break_interval=4):
        self.work_duration = work_duration * 60  # Convert to seconds
        self.short_break_duration = short_break_duration * 60  # Convert to seconds
        self.long_break_duration = long_break_duration * 60  # Convert to seconds
        self.long_break_interval = long_break_interval
        self.current_duration = self.work_duration
        self.current_state = 'work'
        self.session_count = 0
        self.timer_thread = None
        self.running = False

    def start(self):
        if not self.running:
            self.running = 

# Step 4: Cleanup (optional)

In [26]:
# Delete the endpoint

sagemaker_session.delete_endpoint(endpoint_name)